In [1]:
import pandas as pd
import ast
import numpy as np
from haversine import haversine, Unit

# ---------- Load ----------
df = pd.read_csv("../../porto.csv")

# Parse polyline JSON-like strings -> Python lists of [lon, lat]
df["POLYLINE"] = df["POLYLINE"].apply(ast.literal_eval)

# ---------- Derive columns to mirror your SQL CHECKs ----------
# n_points > 0
df["n_points"] = df["POLYLINE"].apply(len)

# start_epoch > 0
df["start_epoch"] = pd.to_numeric(df["TIMESTAMP"], errors="coerce").astype("Int64")

# centroid (domain_middle_lat/lon) from polyline points
def centroid_latlon(points):
    # points are [lon, lat]; return (lat, lon)
    if not points:
        return (np.nan, np.nan)
    lons = [p[0] for p in points]
    lats = [p[1] for p in points]
    return (float(np.mean(lats)), float(np.mean(lons)))

centroids = df["POLYLINE"].apply(centroid_latlon)
df["domain_middle_lat"] = centroids.apply(lambda x: x[0])
df["domain_middle_lon"] = centroids.apply(lambda x: x[1])

# domain_radius > 0 (meters): max haversine distance from centroid to any point
def max_radius_m(points, c_lat, c_lon):
    if not points or np.isnan(c_lat) or np.isnan(c_lon):
        return np.nan
    c = (c_lat, c_lon)  # haversine expects (lat, lon)
    mx = 0.0
    for lon, lat in points:  # note input order [lon, lat]
        d = haversine(c, (lat, lon), unit=Unit.METERS)
        if d > mx:
            mx = d
    return mx

df["domain_radius"] = [
    max_radius_m(pts, lat, lon)
    for pts, lat, lon in zip(df["POLYLINE"], df["domain_middle_lat"], df["domain_middle_lon"])
]

# ---------- Build boolean masks for the CHECK constraints ----------
chk_n_points_positive    = df["n_points"] > 0
chk_start_epoch_positive = df["start_epoch"] > 0
chk_radius_positive      = df["domain_radius"] > 0
chk_latitude_valid       = df["domain_middle_lat"].between(-90, 90, inclusive="both")
chk_longitude_valid      = df["domain_middle_lon"].between(-180, 180, inclusive="both")

valid_mask = (
    chk_n_points_positive
    & chk_start_epoch_positive
    & chk_radius_positive
    & chk_latitude_valid
    & chk_longitude_valid
)

# Optional: see violations
# print({
#     "n_points > 0": (~chk_n_points_positive).sum(),
#     "start_epoch > 0": (~chk_start_epoch_positive).sum(),
#     "domain_radius > 0": (~chk_radius_positive).sum(),
#     "-90<=lat<=90": (~chk_latitude_valid).sum(),
#     "-180<=lon<=180": (~chk_longitude_valid).sum(),
# })

# ---------- Cleaned dataframe ----------
df_clean = df[valid_mask].copy()

In [2]:
#Task 1
num_taxis = df_clean["TAXI_ID"].nunique()
num_trips = len(df_clean)
total_gps_points = df_clean["n_points"].sum()

print(f"Number of taxis: {num_taxis}")
print(f"Number of trips: {num_trips}")
print(f"Total GPS points: {total_gps_points}")

Number of taxis: 442
Number of trips: 1673953
Total GPS points: 83378342


In [3]:
# Task 2
avg_trips_per_taxi = num_trips / num_taxis
print(f"Average trips per taxi: {avg_trips_per_taxi:.2f}")

Average trips per taxi: 3787.22


In [4]:
# Task 3
top_taxis = (
    df_clean.groupby("TAXI_ID")["TRIP_ID"]
    .count()
    .sort_values(ascending=False)
    .head(20)
)

print("Top 20 taxis with most trips:")
print(top_taxis)

Top 20 taxis with most trips:
TAXI_ID
20000403    7848
20000483    7650
20000364    7429
20000307    7311
20000621    7256
20000129    7149
20000492    7125
20000424    7036
20000089    7007
20000529    6922
20000066    6749
20000616    6579
20000678    6484
20000042    6441
20000304    6391
20000235    6391
20000179    6380
20000263    6367
20000325    6367
20000140    6290
Name: TRIP_ID, dtype: int64


In [5]:
# Task 4a
# Most frequent CALL_TYPE per taxi
most_used_call_type = (
    df_clean.groupby("TAXI_ID")["CALL_TYPE"]
    .agg(lambda x: x.value_counts().idxmax())
)

print(most_used_call_type.head(10))

TAXI_ID
20000001    B
20000002    B
20000003    C
20000004    B
20000005    B
20000006    B
20000007    B
20000008    B
20000009    B
20000010    B
Name: CALL_TYPE, dtype: object


In [6]:
# Task 4b
df_clean["trip_duration_s"] = (df_clean["n_points"] - 1) * 15

def trip_distance(points):
    if len(points) < 2:
        return 0.0
    dist = 0.0
    for i in range(1, len(points)):
        lon1, lat1 = points[i-1]
        lon2, lat2 = points[i]
        dist += haversine((lat1, lon1), (lat2, lon2), unit=Unit.METERS)
    return dist

df_clean["trip_distance_m"] = df_clean["POLYLINE"].apply(trip_distance)

df_clean["hour"] = pd.to_datetime(df_clean["TIMESTAMP"], unit="s").dt.hour

def time_band(hour):
    if 0 <= hour < 6:
        return "00–06"
    elif 6 <= hour < 12:
        return "06–12"
    elif 12 <= hour < 18:
        return "12–18"
    else:
        return "18–24"

df_clean["time_band"] = df_clean["hour"].apply(time_band)

# Average duration and distance per call type
avg_stats = df_clean.groupby("CALL_TYPE").agg(
    avg_duration_s=("trip_duration_s", "mean"),
    avg_distance_m=("trip_distance_m", "mean"),
    num_trips=("TRIP_ID", "count")
)

# Share of trips per time band
time_band_share = (
    df_clean.groupby(["CALL_TYPE", "time_band"]).size()
    .groupby(level=0)
    .apply(lambda x: x / x.sum())  # normalize within each CALL_TYPE
    .unstack(fill_value=0)
)

print("=== Average Duration & Distance per CALL_TYPE ===")
print(avg_stats)

print("\n=== Share of Trips per Time Band ===")
print(time_band_share)

=== Average Duration & Distance per CALL_TYPE ===
           avg_duration_s  avg_distance_m  num_trips
CALL_TYPE                                           
A              754.605329     5424.247840     362707
B              672.625683     5108.025765     808818
C              811.725143     6606.950692     502428

=== Share of Trips per Time Band ===
time_band               00–06     06–12     12–18     18–24
CALL_TYPE CALL_TYPE                                        
A         A          0.119256  0.333771  0.321709  0.225264
B         B          0.122367  0.301262  0.344804  0.231566
C         C          0.329331  0.232666  0.240022  0.197981


In [7]:
# Task 5
# --- Aggregate per taxi ---
taxi_usage = df_clean.groupby("TAXI_ID").agg(
    total_hours=("trip_duration_s", lambda x: x.sum() / 3600),  # seconds → hours
    total_distance_km=("trip_distance_m", lambda x: x.sum() / 1000)  # meters → km
)

# Sort by total hours (descending)
taxi_usage_sorted = taxi_usage.sort_values(by="total_hours", ascending=False)

print("Top taxis by total hours driven (with total distance):")
print(taxi_usage_sorted.head(20))

Top taxis by total hours driven (with total distance):
          total_hours  total_distance_km
TAXI_ID                                 
20000904  1961.079167       62788.703932
20000129  1649.758333       36543.724426
20000307  1587.091667       40016.545094
20000529  1507.295833       42389.150091
20000276  1419.883333       47078.202212
20000436  1410.687500       44299.741233
20000483  1397.708333       36174.843834
20000372  1387.133333       40687.944292
20000616  1349.133333       35407.607595
20000179  1324.283333       39841.171776
20000574  1301.925000       37556.280699
20000235  1287.879167       36729.592783
20000621  1282.433333       31987.930963
20000364  1280.491667       41602.180030
20000435  1277.575000       40993.150530
20000199  1270.791667       39565.220840
20000446  1266.633333       35163.690493
20000011  1266.120833       38498.321177
20000395  1263.800000       34208.191882
20000492  1262.566667       33193.583433


In [8]:
# Task 6
city_hall = (41.15794, -8.62911)  # (lat, lon)

def passes_near_city_hall(points, threshold_m=100):
    for lon, lat in points:
        d = haversine((lat, lon), city_hall, unit=Unit.METERS)
        if d <= threshold_m:
            return True
    return False

# Flag trips that pass within 100 m
df_clean["near_city_hall"] = df_clean["POLYLINE"].apply(
    lambda pts: passes_near_city_hall(pts, threshold_m=100)
)

# Extract those trips
trips_near_city_hall = df_clean[df_clean["near_city_hall"]]

print(f"Number of trips near City Hall (<=100m): {len(trips_near_city_hall)}")
print(trips_near_city_hall[["TRIP_ID", "TAXI_ID", "CALL_TYPE", "n_points"]].head(10))


Number of trips near City Hall (<=100m): 126340
                TRIP_ID   TAXI_ID CALL_TYPE  n_points
15  1372637610620000497  20000497         B        64
42  1372638361620000154  20000154         C        29
48  1372641991620000231  20000231         B        12
51  1372636956620000167  20000167         C        32
52  1372641197620000653  20000653         B        24
58  1372642706620000653  20000653         B        16
64  1372636896620000360  20000360         C        43
74  1372642071620000305  20000305         C        65
80  1372637423620000341  20000341         C        50
81  1372639364620000015  20000015         C        42


In [9]:
# Invalid trips: fewer than 3 GPS points
invalid_trips = df_clean[df_clean["n_points"] < 3]

num_invalid_trips = len(invalid_trips)

print(f"Number of invalid trips (< 3 GPS points): {num_invalid_trips}")

Number of invalid trips (< 3 GPS points): 7202


In [10]:
import pandas as pd
import numpy as np
from collections import deque
from haversine import haversine, Unit
from math import cos, radians

# --- Parameters ---
TIME_WINDOW_S = 5
DIST_THRESH_M = 5.0

# --- 1) Explode trips into per-point pings with absolute timestamps ---
#    Each point i occurs at TIMESTAMP + 15*i (Porto dataset sampling)
def explode_points(df):
    records = []
    for row in df.itertuples(index=False):
        start = int(row.TIMESTAMP)
        taxi  = row.TAXI_ID
        pts   = row.POLYLINE  # list of [lon, lat]
        for i, (lon, lat) in enumerate(pts):
            t = start + 15*i
            records.append((t, taxi, lat, lon))
    return pd.DataFrame(records, columns=["time", "taxi_id", "lat", "lon"])

pings = explode_points(df_clean)
# Keep only pings from trips with at least 1 point (df_clean already enforces >0 and > radius)
pings = pings.dropna(subset=["time", "taxi_id", "lat", "lon"]).astype({"time":"int64"})
pings.sort_values("time", inplace=True, ignore_index=True)

# --- 2) Sliding time window over sorted pings ---
# Keep a deque of points within the last TIME_WINDOW_S
# Use a quick bounding box prefilter to avoid many haversine calls.
# At Porto's latitude (~41°), 1° lat ~ 111,320 m; 1° lon ~ 111,320*cos(lat) ≈ 84,000 m.
LAT_DEG_PER_M = 1.0 / 111_320.0
def lon_deg_per_m_at(lat):
    return 1.0 / (111_320.0 * max(0.1, cos(radians(lat))))  # guard cos->0 near poles (not needed here but safe)

window = deque()  # items: (time, taxi_id, lat, lon)
encounters = []   # (taxi_a, taxi_b, time, distance_m)

for row in pings.itertuples(index=False):
    t, taxi, lat, lon = row.time, row.taxi_id, row.lat, row.lon

    # evict old points
    while window and (t - window[0][0]) > TIME_WINDOW_S:
        window.popleft()

    # precompute lon degree threshold at current latitude
    lat_eps = DIST_THRESH_M * LAT_DEG_PER_M
    lon_eps = DIST_THRESH_M * lon_deg_per_m_at(lat)

    # compare to points in the window
    for t2, taxi2, lat2, lon2 in window:
        if taxi2 == taxi:
            continue
        # cheap prefilter: bounding box within ~5m
        if abs(lat - lat2) <= lat_eps and abs(lon - lon2) <= lon_eps:
            d = haversine((lat, lon), (lat2, lon2), unit=Unit.METERS)
            if d <= DIST_THRESH_M:
                a, b = (taxi, taxi2) if taxi < taxi2 else (taxi2, taxi)
                encounters.append((a, b, min(t, t2), d))

    # add current point to window
    window.append((t, taxi, lat, lon))

# --- 3) Aggregate to unique taxi pairs that met the criteria at least once ---
if encounters:
    enc_df = pd.DataFrame(encounters, columns=["taxi_a", "taxi_b", "time", "distance_m"])

    pairs = (
        enc_df.groupby(["taxi_a", "taxi_b"])
        .agg(
            first_time=("time", "min"),
            last_time=("time", "max"),
            encounters=("time", "count"),
            min_distance_m=("distance_m", "min"),
        )
        .reset_index()
        .sort_values(["first_time", "taxi_a", "taxi_b"], ascending=[True, True, True])
    )

    print(f"Unique taxi pairs within 5m & 5s at least once: {len(pairs)}")
    print(pairs.head(20))  # show a sample
else:
    print("No pairs found within 5 meters and 5 seconds.")


Unique taxi pairs within 5m & 5s at least once: 44998
         taxi_a    taxi_b  first_time   last_time  encounters  min_distance_m
37798  20000423  20000451  1372638171  1372638171           1        3.019350
30849  20000309  20000653  1372640095  1397574241           5        0.753350
20258  20000171  20000671  1372649094  1401482847           7        2.001511
30533  20000307  20000534  1372649316  1372649316           1        1.505504
37032  20000398  20000540  1372649899  1395655386           4        1.807443
19988  20000167  20000403  1372653680  1401801227           5        0.000000
15974  20000128  20000154  1372653763  1402511283           2        3.175811
31945  20000325  20000571  1372655214  1402808434           2        2.504273
26200  20000247  20000514  1372656391  1400574563           3        1.809050
43101  20000561  20000570  1372656721  1372656751           3        1.507145
33035  20000337  20000539  1372657141  1373015256           3        3.014654
4480   200

In [11]:
# If you already have df_clean with parsed POLYLINE and n_points, you can skip the prep block.
df_use = df_clean.copy() if 'df_clean' in globals() else pd.read_csv("../../porto.csv")
if 'POLYLINE' in df_use.columns and df_use['POLYLINE'].dtype == object and not isinstance(df_use['POLYLINE'].iloc[0], list):
    df_use['POLYLINE'] = df_use['POLYLINE'].apply(ast.literal_eval)
if 'n_points' not in df_use.columns:
    df_use['n_points'] = df_use['POLYLINE'].apply(len)

# Start/end timestamps (seconds since epoch). Porto dataset uses 15s spacing between points.
df_use['end_epoch'] = df_use['TIMESTAMP'] + (df_use['n_points'] - 1) * 15

# Convert to local Porto time for calendar-day comparison
start_dt = pd.to_datetime(df_use['TIMESTAMP'], unit='s', utc=True).dt.tz_convert('Europe/Lisbon')
end_dt   = pd.to_datetime(df_use['end_epoch'], unit='s', utc=True).dt.tz_convert('Europe/Lisbon')

# Midnight crosser: started on one date, ended on a different date
midnight_mask = start_dt.dt.date != end_dt.dt.date

midnight_crossers = df_use.loc[midnight_mask, [
    'TRIP_ID', 'TAXI_ID', 'CALL_TYPE', 'TIMESTAMP', 'n_points', 'end_epoch'
]].copy()
midnight_crossers['start_local'] = start_dt[midnight_mask]
midnight_crossers['end_local']   = end_dt[midnight_mask]
midnight_crossers['duration_s']  = (midnight_crossers['end_epoch'] - midnight_crossers['TIMESTAMP']).astype('int64')

print(f"Midnight-crossing trips: {len(midnight_crossers)}")
print(midnight_crossers.head(20))

Midnight-crossing trips: 8029
                  TRIP_ID   TAXI_ID CALL_TYPE   TIMESTAMP  n_points  \
4236  1372718798620000370  20000370         A  1372718798        62   
4266  1372719200620000031  20000031         B  1372719200        54   
4288  1372719157620000632  20000632         B  1372719157        33   
4307  1372719271620000503  20000503         B  1372719271        35   
4322  1372719469620000409  20000409         B  1372719469        19   
4341  1372719230620000406  20000406         A  1372719230        59   
4344  1372719093620000022  20000022         B  1372719093        43   
4358  1372719315620000008  20000008         B  1372719315        41   
4364  1372718162620000560  20000560         C  1372718162       105   
4399  1372719396620000304  20000304         C  1372719396        69   
4407  1372719050620000596  20000596         B  1372719050        60   
4781  1372719317620000101  20000101         C  1372719317        69   
5525  1372717190620000388  20000388         A  

In [12]:
# Require at least two points to compare start/end
mask_ge2 = df_clean["n_points"] >= 2

# Extract start/end (lat, lon) from POLYLINE
def start_latlon(pts):
    lon, lat = pts[0]
    return lat, lon

def end_latlon(pts):
    lon, lat = pts[-1]
    return lat, lon

df_use = df_clean[mask_ge2].copy()
df_use["start_latlon"] = df_use["POLYLINE"].apply(start_latlon)
df_use["end_latlon"]   = df_use["POLYLINE"].apply(end_latlon)

# Haversine distance (meters) between start and end
df_use["start_end_dist_m"] = df_use.apply(
    lambda r: haversine(r["start_latlon"], r["end_latlon"], unit=Unit.METERS),
    axis=1
)

# Circular trips: start/end within 50 m
circular_mask = df_use["start_end_dist_m"] <= 50.0
circular_trips = df_use.loc[circular_mask, [
    "TRIP_ID", "TAXI_ID", "CALL_TYPE", "n_points", "start_end_dist_m"
]].sort_values("start_end_dist_m")

print(f"Circular trips (start/end ≤ 50 m): {len(circular_trips)}")
print(circular_trips.head(20))


Circular trips (start/end ≤ 50 m): 25386
                     TRIP_ID   TAXI_ID CALL_TYPE  n_points  start_end_dist_m
1489162  1400391940620000492  20000492         C         4               0.0
765005   1386858221620000172  20000172         C         3               0.0
1529634  1401118088620000066  20000066         C         3               0.0
1309482  1397251581620000060  20000060         A        68               0.0
1498168  1400576225620000525  20000525         C        99               0.0
508386   1382110717620000099  20000099         C         6               0.0
269858   1377966324620000372  20000372         A        77               0.0
1623940  1402682991620000565  20000565         B         4               0.0
1064643  1392644995620000448  20000448         C         3               0.0
1011846  1391674467620000596  20000596         A        37               0.0
735851   1386331208620000032  20000032         B         3               0.0
316555   1378891786620000303  20000

In [13]:
# Compute trip end time (epoch seconds)
df_clean["end_epoch"] = df_clean["TIMESTAMP"] + (df_clean["n_points"] - 1) * 15

# Sort trips per taxi chronologically
df_sorted = df_clean.sort_values(["TAXI_ID", "TIMESTAMP"]).copy()

# Shift start times to align consecutive trips
df_sorted["next_start"] = df_sorted.groupby("TAXI_ID")["TIMESTAMP"].shift(-1)

# Idle time = next trip start - current trip end
df_sorted["idle_time_s"] = df_sorted["next_start"] - df_sorted["end_epoch"]

# Keep only positive idle times (ignore overlaps / errors)
df_sorted = df_sorted[df_sorted["idle_time_s"] > 0]

# Average idle time per taxi (in hours for readability)
avg_idle = (
    df_sorted.groupby("TAXI_ID")["idle_time_s"]
    .mean()
    .sort_values(ascending=False)
    .head(20) / 3600  # convert seconds → hours
)

print("Top 20 taxis with highest average idle time (hours):")
print(avg_idle)

Top 20 taxis with highest average idle time (hours):
TAXI_ID
20000941    1746.613519
20000969     233.437701
20000312      13.070253
20000510      12.813855
20000609      11.835517
20000579      11.384354
20000079       9.603897
20000072       9.108515
20000185       9.008618
20000449       8.857682
20000902       7.532595
20000170       6.733472
20000535       6.625844
20000315       6.445517
20000071       6.240770
20000225       6.216521
20000545       5.911245
20000407       5.630359
20000205       5.564613
20000443       5.411748
Name: idle_time_s, dtype: float64
